# Polymarket API Python

**Resources:**
- [Polymarket API Docs](https://docs.polymarket.com/)
- [Gamma API (Market Data)](https://gamma-api.polymarket.com/)
- [Data API (User Data)](https://data-api.polymarket.com/)
- [py-clob-client (3rd Party using Official SDK)](https://github.com/Polymarket/py-clob-client)

## 1. Setup

In [2]:
# pip install py-clob-client requests

In [3]:
import requests
import time
import json
from pprint import pprint

from py_clob_client.client import ClobClient
from py_clob_client.clob_types import OrderArgs, MarketOrderArgs, OrderType, OpenOrderParams, BalanceAllowanceParams, AssetType
from py_clob_client.order_builder.constants import BUY, SELL

GAMMA_API = "https://gamma-api.polymarket.com"
DATA_API = "https://data-api.polymarket.com"
CLOB_API = "https://clob.polymarket.com"

## 2. Market Discovery

In [40]:
# Fetch active markets sorted by volume
response = requests.get(
    f"{GAMMA_API}/markets",
    params={
        "limit": 1000,
        "active": True,
        "closed": False,
        "order": "volume24hr",
        "ascending": False
    }
)
markets = response.json()
print(f"Found {len(markets)} markets\n")

Found 500 markets



In [41]:
for m in markets[:5]:
    print(f"Question: {m['question']}")
    print(f"  Volume 24h: ${m.get('volume24hr', 0):,.0f}")
    print(f"  Liquidity: ${m.get('liquidityNum', 0):,.0f}")
    print(f"  Prices: {m.get('outcomePrices', 'N/A')}")
    print(f"end date: {m['endDate']}")
    print()

Question: Fed increases interest rates by 25+ bps after January 2026 meeting?
  Volume 24h: $11,065,623
  Liquidity: $7,110,259
  Prices: ["0.0035", "0.9965"]
end date: 2026-01-28T00:00:00Z

Question: Fed decreases interest rates by 50+ bps after January 2026 meeting?
  Volume 24h: $6,954,493
  Liquidity: $2,401,094
  Prices: ["0.0065", "0.9935"]
end date: 2026-01-28T00:00:00Z

Question: Will Meituan have the best AI model at the end of January 2026?
  Volume 24h: $2,626,216
  Liquidity: $75,351
  Prices: ["0.0005", "0.9995"]
end date: 2026-01-31T00:00:00Z

Question: Fed decreases interest rates by 25 bps after January 2026 meeting?
  Volume 24h: $1,748,558
  Liquidity: $1,034,397
  Prices: ["0.065", "0.935"]
end date: 2026-01-28T00:00:00Z

Question: Will Trump nominate Barron Trump as the next Fed chair?
  Volume 24h: $1,720,517
  Liquidity: $166,480
  Prices: ["0.0005", "0.9995"]
end date: 2026-12-31T00:00:00Z



## 3. Market Deep Dive

In [49]:
market = markets[1]
market['outcomePrices']

'["0.0065", "0.9935"]'

In [7]:
print(f"Market: {market['question']}")
print(f"End Date: {market['endDate']}")
print(f"Condition ID: {market['conditionId']}")

Market: Fed decreases interest rates by 50+ bps after January 2026 meeting?
End Date: 2026-01-28T00:00:00Z
Condition ID: 0x17815081230e3b9c78b098162c33b1ffa68c4ec29c123d3d14989599e0c2e113


In [8]:
clob_token_ids = market.get('clobTokenIds')
clob_token_ids = json.loads(clob_token_ids)
print(f"Token IDs: {clob_token_ids}")

if len(clob_token_ids) >= 2:
    yes_token_id = clob_token_ids[0]
    no_token_id = clob_token_ids[1]
    print(f"YES token: {yes_token_id}")
    print(f"NO token: {no_token_id}")

Token IDs: ['11862165566757345985240476164489718219056735011698825377388402888080786399275', '71478852790279095447182996049071040792010759617668969799049179229104800573786']
YES token: 11862165566757345985240476164489718219056735011698825377388402888080786399275
NO token: 71478852790279095447182996049071040792010759617668969799049179229104800573786


## 4. Order Book Analysis

In [9]:
# Read-only client for order book
client = ClobClient(CLOB_API)

In [10]:
# Fetch order book for YES token
if yes_token_id:
    book = client.get_order_book(yes_token_id)
    
    sorted_bids = sorted(book.bids, key=lambda x: float(x.price), reverse=True)
    sorted_asks = sorted(book.asks, key=lambda x: float(x.price), reverse=False)
    
    print("=== YES Token Order Book ===")
    print(f"\nTop 5 Asks (sell orders):")
    for ask in sorted_asks[:5]:
        print(f"  Price: {ask.price} | Size: {ask.size}")
        
    print(f"\nTop 5 Bids (buy orders):")
    for bid in sorted_bids[:5]:
        print(f"  Price: {bid.price} | Size: {bid.size}")


=== YES Token Order Book ===

Top 5 Asks (sell orders):
  Price: 0.007 | Size: 2047339.48
  Price: 0.008 | Size: 43658
  Price: 0.009 | Size: 3377
  Price: 0.01 | Size: 15090.78
  Price: 0.011 | Size: 27809.37

Top 5 Bids (buy orders):
  Price: 0.006 | Size: 221949.03
  Price: 0.005 | Size: 2748463.82
  Price: 0.004 | Size: 1689498.98
  Price: 0.003 | Size: 1523660.5
  Price: 0.002 | Size: 2104144.81


In [11]:
# Get midpoint, prices, and spread
mid = client.get_midpoint(yes_token_id)
buy_price = client.get_price(yes_token_id, side="BUY")
sell_price = client.get_price(yes_token_id, side="SELL")
spread = client.get_spread(yes_token_id)

print(f"Midpoint: {mid['mid']}")
print(f"Best ask (buy at): {buy_price['price']}")
print(f"Best bid (sell at): {sell_price['price']}")
print(f"Spread: {spread['spread']}")

Midpoint: 0.0065
Best ask (buy at): 0.006
Best bid (sell at): 0.007
Spread: 0.001


## 5. Authentication

In [12]:
# Your credentials (keep these secret!)
FUNDER_ADDRESS = "your-wallet-address"  # For proxy wallets
PRIVATE_KEY = "your-private-key-here"  # Never commit this

# Signature types:
# 0 = EOA (MetaMask, hardware wallet)
# 1 = Email/Magic wallet
# 2 = Browser wallet proxy
SIGNATURE_TYPE = 1

In [13]:
# Initialize authenticated client
auth_client = ClobClient(
    CLOB_API,
    key=PRIVATE_KEY,
    chain_id=137,
    signature_type=SIGNATURE_TYPE,
    funder=FUNDER_ADDRESS
)

# Get & Set API credentials
creds = auth_client.derive_api_key()
auth_client.set_api_creds(creds)

Error: Non-hexadecimal digit found

In [ ]:
balance = auth_client.get_balance_allowance(BalanceAllowanceParams(asset_type=AssetType.COLLATERAL))
usdc_balance = int(balance['balance']) / 1e6
print(f"USDC Balance: ${usdc_balance:.2f}")

## 6. Place a Market Order

In [ ]:
# Create a market order: Spend $5 on YES shares at market price
market_order = MarketOrderArgs(
    token_id=yes_token_id,
    amount=5.0,  # Dollar amount to spend
    side=BUY,
    order_type=OrderType.FOK  # Fill-or-Kill
)

signed_market_order = auth_client.create_market_order(market_order)
print("Market order signed!")

In [ ]:
# Execute market order
response = auth_client.post_order(signed_market_order, OrderType.FOK)
print("Market order executed!")
pprint(response)

## 7. Place a Limit Order

In [ ]:
# Create a limit order: Buy 10 YES shares at $0.50
limit_order = OrderArgs(
    token_id=yes_token_id,
    price=0.001,  # Price per share
    size=10.0,   # Number of shares
    side=BUY
)

# Sign the order
signed_order = auth_client.create_order(limit_order)
print("Order signed!")

In [ ]:
# Post the order (GTC = Good Till Cancelled)
response = auth_client.post_order(signed_order, OrderType.GTC)
print(f"Order placed!")
pprint(response)

## 8. Open Orders

In [ ]:
# Get your open orders
open_orders = auth_client.get_orders(OpenOrderParams())
print(f"Open orders: {len(open_orders)}")

for order in open_orders[:5]:
    print(f"  ID: {order['id'][:20]}...")
    print(f"  Side: {order['side']} | Price: {order['price']} | Size: {order['original_size']}")
    print()

## 9. Cancel Orders

In [ ]:
# Cancel a specific order
open_orders = auth_client.get_orders(OpenOrderParams())
if open_orders:
    order_id = open_orders[0]["id"]
    result = auth_client.cancel(order_id)
    print(f"Cancelled order: {order_id[:20]}...")
    pprint(result)

In [ ]:
# Cancel all orders
result = auth_client.cancel_all()
print("All orders cancelled!")
pprint(result)

## 10. BONUS 1: Price Tracker

In [14]:
def track_price(token_id, duration_seconds=30, interval=5):
    """Track price changes in real-time."""
    print(f"Tracking price for {duration_seconds}s...\n")

    client = ClobClient(CLOB_API)
    start_time = time.time()
    prices = []

    while time.time() - start_time < duration_seconds:
        mid = client.get_midpoint(token_id)
        mid_price = float(mid['mid'])
        timestamp = time.strftime("%H:%M:%S")
        prices.append(mid_price)

        change = ""
        if len(prices) > 1:
            diff = prices[-1] - prices[-2]
            change = f" ({'+' if diff >= 0 else ''}{diff:.4f})"

        print(f"[{timestamp}] Price: {mid_price}{change}")
        time.sleep(interval)

    print(f"\nTotal change: {prices[-1] - prices[0]:.4f}")
    return prices

In [15]:
# Run the tracker
prices = track_price(yes_token_id, duration_seconds=10, interval=1)

Tracking price for 10s...

[19:34:53] Price: 0.0065
[19:34:54] Price: 0.0065 (+0.0000)
[19:34:55] Price: 0.0065 (+0.0000)
[19:34:56] Price: 0.0065 (+0.0000)
[19:34:57] Price: 0.0065 (+0.0000)
[19:34:58] Price: 0.0065 (+0.0000)
[19:34:59] Price: 0.0065 (+0.0000)
[19:35:00] Price: 0.0065 (+0.0000)
[19:35:01] Price: 0.0065 (+0.0000)
[19:35:02] Price: 0.0065 (+0.0000)

Total change: 0.0000


## 11. BONUS 2: Address Position Tracker

In [ ]:
def get_user_positions(wallet_address):
    """Get a user's current positions."""
    url = f"{DATA_API}/positions"
    params = {"user": wallet_address}
    response = requests.get(url, params=params)
    return response.json()


In [18]:
address = "0x6a72f61820b26b1fe4d956e17b6dc2a1ea3033ee"
positions = get_user_positions(address)
positions

[{'proxyWallet': '0x6a72f61820b26b1fe4d956e17b6dc2a1ea3033ee',
  'asset': '105953956915150002522974165882693825582092111865946642103065245868242929489995',
  'conditionId': '0x2a496f947e51993cdb4af43c8dd851780a8bac86c06cd7ad5dcd5741f46ce843',
  'size': 1213870.438533,
  'avgPrice': 0.028785,
  'initialValue': 34941.2605731724,
  'currentValue': 0,
  'cashPnl': -34941.26057317241,
  'percentPnl': -100,
  'totalBought': 1213870.438533,
  'realizedPnl': 0,
  'percentRealizedPnl': -99.99999999999997,
  'curPrice': 0,
  'redeemable': True,
  'mergeable': False,
  'title': 'Will Jake Paul vs. Anthony Joshua have no official winner?',
  'slug': 'will-the-fight-end-in-another-outcome',
  'icon': 'https://polymarket-upload.s3.us-east-2.amazonaws.com/boxing-jake-paul-vs-anthony-joshua-third-option-included-O9JsNln3VFfX.jpg',
  'eventId': '85540',
  'eventSlug': 'boxing-jake-paul-vs-anthony-joshua-third-option-included',
  'outcome': 'Yes',
  'outcomeIndex': 0,
  'oppositeOutcome': 'No',
  'oppos